In [59]:
import spacy
import spacy_dbpedia_spotlight
import numpy as np
from bs4 import BeautifulSoup
import requests

In [60]:
nlp = spacy.load('en_core_web_lg')

nlp.add_pipe('dbpedia_spotlight',last=True)

In [138]:
text = "Gary Neville criticises Paul Pogba a lot, but not Jack Grealish."
doc = nlp(text)

In [139]:
entities=[]

for ent in doc.ents:
    URL=ent._.dbpedia_raw_result['@URI']
    start=int(ent._.dbpedia_raw_result['@offset'])
    end=int(ent._.dbpedia_raw_result['@offset'])+len(ent._.dbpedia_raw_result['@surfaceForm'])
    ent_type=ent._.dbpedia_raw_result['@types'].split(",")[-1]
    entities.append([URL,start,end,ent_type])

    


In [140]:
for ent in entities:
    print(ent)

['http://dbpedia.org/resource/Gary_Neville', 0, 12, 'DBpedia:SoccerManager']
['http://dbpedia.org/resource/Paul_Pogba', 24, 34, 'DBpedia:SoccerPlayer']
['http://dbpedia.org/resource/Jack_Grealish', 50, 63, 'DBpedia:SoccerPlayer']


In [141]:
abstracts=[]

for ent in entities:
    URL=ent[0]
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find('p', class_="lead").getText()
    if content not in abstracts:
        abstracts.append(content)

In [142]:
for a in abstracts:
    print(a,"\n")

Gary Alexander Neville (born 18 February 1975) is an English football pundit, coach, and former player who is a co-owner of Salford City. After retiring from football in 2011, Neville went into punditry and was a commentator for Sky Sports, until he took over the head coach position at Valencia in 2015. After being sacked by the club in 2016, he returned to his position as a pundit for Sky Sports later that year. He was also assistant manager for the England national team from 2012 to 2016. 

Paul Labile Pogba (born 15 March 1993) is a French professional footballer who plays for Premier League club Manchester United and the France national team. He operates primarily as a central midfielder, but can also be deployed as a left winger, attacking midfielder, defensive midfielder and deep-lying playmaker. 

Jack Peter Grealish (born 10 September 1995) is a professional footballer who plays as a winger or attacking midfielder for Premier League club Manchester City and the England national